## 1. Las top 10 fechas donde hay más tweets más el usuario (username) que más publicaciones tiene por cada uno de esos días.

### Estructura del notebook: 
1. Explicación detallada del código
2. Revisión de tiempos de ejecución y memoria de q1_time.py y q1_memory.py

In [89]:
import pandas as pd

In [35]:
def extract_user_id(dicc: dict):
    """ Función específicamente creada para usar junto al método apply, aplicado a una columna de un
    dataframe de pandas.
    Extrae el id del usuario que viene dentro de la columna user en los datos recibidos.

    Args:
        dicc (dict): diccionario de los datos del usuario.

    Returns:
        int: retorna el id del usuario. 
    """
    return dicc.get('id')

def extract_username(dicc: dict):
    """Función específicamente creada para usar junto al método apply, aplicado a una columna de un
    dataframe de pandas.
    Extrae el username del usuario que viene dentro de la columna user en los datos recibidos.

    Args:
        dicc (dict): diccionario de los datos del usuario.

    Returns:
        str: retorna el username del usuario.
    """
    return dicc.get('username')

### Creamos función con la libreria jsonlines para la lectura de los datos

In [90]:
import jsonlines

def read_json_lines(file_path: str, cols: list):
    """Lee un archivo en el cual hay un objeto JSON por línea y devuelve los datos solicitados en un dataframe de pandas.

    Args:
        file_path (str): path del archivo a leer..
        cols (list): lista de "keys" de cada JSON que queremos devolver

    Returns:
        pd.DataFrame: dataframe de pandas con los datos solicitados.
    """
    data = []
    with jsonlines.open(file_path) as reader:
        for item in reader:
            item = {key: item[key] for key in cols if key in item}
            data.append(item)
    return pd.DataFrame(data)

### Visualizamos los datos

In [83]:
# Lectura de los datos. La estructura de los datos son un objeto JSON por cada fila.
file_path = '../farmers-protest-tweets-2021-2-4.json'
cols = ['date', 'id', 'user']
df = read_json_lines(file_path, cols)
df.head()

,date,id,user
0,2021-02-24T09:23:35+00:00,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':..."
1,2021-02-24T09:23:32+00:00,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra..."
2,2021-02-24T09:23:22+00:00,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ..."
3,2021-02-24T09:23:16+00:00,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '..."
4,2021-02-24T09:23:10+00:00,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre..."


### Así se ven las columnas que utilizaremos

In [84]:
# Se agregan las columnas utilizadas para calcular los usernames que tweets realizaron.
df['user_id'] = df['user'].apply(extract_user_id)
df['username'] = df['user'].apply(extract_username)

#Convertimos las fechas de tipo datetime a date 
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df[['date', 'id', 'user_id', 'username']].head()

,date,id,user_id,username
0,2021-02-24,1364506249291784198,45091142,ArjunSinghPanam
1,2021-02-24,1364506237451313155,1355092620662329349,PrdeepNain
2,2021-02-24,1364506195453767680,476006247,parmarmaninder
3,2021-02-24,1364506167226032128,137908912,anmoldhaliwal
4,2021-02-24,1364506144002088963,1358786390696206337,KotiaPreet


### Obtenemos las top 10 fechas con más tweets y filtramos nuestro df por esas fechas

In [85]:
top_10_dates = df['date'].value_counts().nlargest(10).index
df = df.loc[df.date.isin(top_10_dates)]
top_10_dates

Index([2021-02-12, 2021-02-13, 2021-02-17, 2021-02-16, 2021-02-14, 2021-02-18,
       2021-02-15, 2021-02-20, 2021-02-23, 2021-02-19],
      dtype='object', name='date')

### Creamos una columna llamada "rnk" (rank) para saber cuántos tweets realizó cada usuario en cada día.

In [86]:
df['rnk'] = df.groupby(['date', 'user_id']).cumcount()
df.sort_values('rnk', ascending=False)

,date,id,user,user_id,username,rnk
40498,2021-02-19,1362639704709992448,"{'username': 'Preetm91', 'displayname': 'ਪ੍ਰੀਤ...",1344090487087575045,Preetm91,266
40497,2021-02-19,1362639706526150657,"{'username': 'Preetm91', 'displayname': 'ਪ੍ਰੀਤ...",1344090487087575045,Preetm91,265
40496,2021-02-19,1362639708178669572,"{'username': 'Preetm91', 'displayname': 'ਪ੍ਰੀਤ...",1344090487087575045,Preetm91,264
40495,2021-02-19,1362639709915148291,"{'username': 'Preetm91', 'displayname': 'ਪ੍ਰੀਤ...",1344090487087575045,Preetm91,263
40494,2021-02-19,1362639711789981700,"{'username': 'Preetm91', 'displayname': 'ਪ੍ਰੀਤ...",1344090487087575045,Preetm91,262
...,...,...,...,...,...,...
64758,2021-02-16,1361777714563227652,"{'username': 'gawdbrar', 'displayname': 'Gurpy...",1334959942315339776,gawdbrar,0
64762,2021-02-16,1361777445477695488,"{'username': 'Jaito_Rajinder', 'displayname': ...",848236856617598982,Jaito_Rajinder,0
64763,2021-02-16,1361777440083697665,"{'username': 'pawan_jyot', 'displayname': 'Paw...",3016698860,pawan_jyot,0
64766,2021-02-16,1361777135363497986,"{'username': 'Lambardar707', 'displayname': 'S...",1268551066343419909,Lambardar707,0


### Creamos un nuevo dataframe "df2" en el cual realizamos los siguientes pasos: 

1. Agrupamos por date y user_id y nos quedamos con el máximo valor de rnk. De esta forma obtenemos el usuario que mas veces twiteo por día.

ACLARACIÓN IMPORTANTE: podría haber usado el username y no el user_id para ahorrarme un paso, pero no sería una buena práctica ya que el username no es un identificador único y twitter permite cambiarlo.

2. Ordenamos el dataframe de mayor a menor y aplicamos un first() por date. Esto nos permite quedarnos con solo un registro por fecha.

3. En este paso hacemos un merge con la columna username para obtener el username del usuario que realizó más tweets. Además, por las dudas de que algún usuario haya cambiado su username entre esas fechas y obtengamos registros duplicados, eliminamos el registro duplicado y nos quedamos con el username más reciente (esto lo hacemos con el keep='first').

In [74]:
df2 = df[['date', 'user_id', 'rnk']].\
                    groupby(['date', 'user_id'], as_index=False).max().\
                    sort_values('rnk', ascending=False).\
                    groupby(['date'], as_index=False).first().\
                    sort_values('rnk', ascending=False).\
                    merge(df[['user_id', 'username']].drop_duplicates(keep='first'), how='left', on='user_id')

df2

,date,user_id,rnk,username
0,2021-02-19,1344090487087575045,266,Preetm91
1,2021-02-18,1355443675631783936,194,neetuanjle_nitu
2,2021-02-17,1334454542713294849,184,RaaJVinderkaur
3,2021-02-13,1333757109842939915,177,MaanDee08215437
4,2021-02-12,1332853294851530753,175,RanbirS00614606
5,2021-02-23,217105273,134,Surrypuria
6,2021-02-15,452391771,133,jot__b
7,2021-02-16,452391771,132,jot__b
8,2021-02-14,1354532795847073796,118,rebelpacifist
9,2021-02-20,1342840568666189825,107,MangalJ23056160


### El método to_records() nos permite obtener un array de tuplas de las columnas del dataframe.

In [75]:
q1_records = df2.to_records(index=False)
q1_records

rec.array([(datetime.date(2021, 2, 19), 1344090487087575045, 266, 'Preetm91'),
           (datetime.date(2021, 2, 18), 1355443675631783936, 194, 'neetuanjle_nitu'),
           (datetime.date(2021, 2, 17), 1334454542713294849, 184, 'RaaJVinderkaur'),
           (datetime.date(2021, 2, 13), 1333757109842939915, 177, 'MaanDee08215437'),
           (datetime.date(2021, 2, 12), 1332853294851530753, 175, 'RanbirS00614606'),
           (datetime.date(2021, 2, 23),           217105273, 134, 'Surrypuria'),
           (datetime.date(2021, 2, 15),           452391771, 133, 'jot__b'),
           (datetime.date(2021, 2, 16),           452391771, 132, 'jot__b'),
           (datetime.date(2021, 2, 14), 1354532795847073796, 118, 'rebelpacifist'),
           (datetime.date(2021, 2, 20), 1342840568666189825, 107, 'MangalJ23056160')],
          dtype=[('date', 'O'), ('user_id', '<i8'), ('rnk', '<i8'), ('username', 'O')])

### Retornamos solo las columnas solicitadas en el ejercicio.

In [76]:
[(row.date, row.username) for row in q1_records]

[(datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160')]

----
## Repaso de tiempos de ejecución de q1_time.py y q1_memory.py

### Tiempo de ejecución de la función q1_time y q1_memory, nombrada como q1

In [81]:
import cProfile

def q1(file_path: str):

    df = read_json_lines(file_path, cols=['date', 'user', 'id'])

    df['user_id'] = df['user'].apply(extract_user_id)
    df['username'] = df['user'].apply(extract_username)

    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.date

    top_10_dates = df['date'].value_counts().nlargest(10).index
    df = df.loc[df.date.isin(top_10_dates)]

    df['rnk'] = df.groupby(['date', 'user_id']).cumcount()

    df2 = df[['date', 'user_id', 'rnk']].\
                        groupby(['date', 'user_id'], as_index=False).max().\
                        sort_values('rnk', ascending=False).\
                        groupby(['date'], as_index=False).first().\
                        sort_values('rnk', ascending=False).\
                        merge(df[['user_id', 'username']].drop_duplicates(keep='first'), how='left', on='user_id')

    q1_records = df2.to_records(index=False)
    
    return [(row.date, row.username) for row in q1_records]

cProfile.run("q1('../farmers-protest-tweets-2021-2-4.json')")


         2983048 function calls (2982726 primitive calls) in 2.916 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.035    0.035    2.853    2.853 3072196544.py:11(q1_time)
        1    0.099    0.099    2.390    2.390 3072196544.py:2(read_json_lines)
        1    0.000    0.000    0.000    0.000 3072196544.py:35(<listcomp>)
   117407    0.033    0.000    0.033    0.000 3072196544.py:6(<dictcomp>)
   117407    0.013    0.000    0.040    0.000 4170275938.py:1(extract_user_id)
   117407    0.013    0.000    0.040    0.000 4170275938.py:14(extract_username)
        1    0.000    0.000    0.000    0.000 <attrs generated init jsonlines.jsonlines.Reader>:1(__init__)
       91    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.020  

### Uso de memoria de q1

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
    43     94.5 MiB     94.5 MiB           1   @profile
    44                                         def q1_time(file_path: str):
    45                                         
    46    482.6 MiB    388.1 MiB           1       df = read_json_lines(file_path, cols=['date', 'user', 'id'])
    47                                         
    48    488.3 MiB      5.7 MiB           1       df['user_id'] = df['user'].apply(extract_user_id)
    49    488.7 MiB      0.4 MiB           1       df['username'] = df['user'].apply(extract_username)
    50                                         
    51    492.1 MiB      3.5 MiB           1       df['date'] = pd.to_datetime(df['date'])
    52    496.6 MiB      4.5 MiB           1       df['date'] = df['date'].dt.date
    53                                         
    54    497.0 MiB      0.4 MiB           1       top_10_dates = df['date'].value_counts().nlargest(10).index
    55    469.6 MiB    -27.4 MiB           1       df = df.loc[df.date.isin(top_10_dates)]
    56                                         
    57    475.0 MiB      5.4 MiB           1       df['rnk'] = df.groupby(['date', 'user_id']).cumcount()
    58                                         
    59    479.6 MiB      0.0 MiB           2       df2 = df[['date', 'user_id', 'rnk']].\
    60    479.2 MiB      4.3 MiB           1                           groupby(['date', 'user_id'], as_index=False).max().\
    61    479.2 MiB      0.0 MiB           1                           sort_values('rnk', ascending=False).\
    62    479.3 MiB      0.1 MiB           1                           groupby(['date'], as_index=False).first().\
    63    479.3 MiB      0.0 MiB           1                           sort_values('rnk', ascending=False).\
    64    479.6 MiB      0.3 MiB           1                           merge(df[['user_id', 'username']].drop_duplicates(keep='first'), how='left', on='user_id')
    65                                         
    66    479.6 MiB      0.0 MiB           1       q1_records = df2.to_records(index=False)
    67                                             
    68    479.6 MiB      0.0 MiB          13       return [(row.date, row.username) for row in q1_records]

## Conclusiones:

- En este ejercicio, a diferencia de los demás, q1_time y q1_memory quedaron iguales. En principio, se utilizaba pd.read_json() para leer los datos pero luego de implementar la librería jsonlines, el consumo de memoria y la velocidad mejoró significativamente.
- Opté por no implementar un enfoque en particiones (chuncksize) como en los ejercicios 2 y 3 debido a que en este ejercicio es necesario hacer varios groupby en todo el dataset.
- La creación del dataframe "df2" tiene potencial para ser mejorada en rendimiento y velocidad.